In [1]:
!pip install pypdf
!pip install pdfplumber
!pip install PyPDF4

In [2]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("sample/jd_1.pdf")
pages = loader.load_and_split()

In [3]:
len(pages)

2

In [5]:
pages[0]

Document(page_content="About  us FULL  STACK  ENGINEER  \n \nQode is dedicated to helping technical talent around the world find meaningful careers \nthat match their skills and interests. Our platform provides a range of resources and \ntools that empower job seekers to take control of their careers and conne ct with top \nemployers across a variety of industries. We believe that every individual deserves to \nfind work that they're passionate about, and we are committed to making that vision a \nreality.  \n \nQode's team of experienced professionals is passionate about  creating a better world of \nwork by providing innovative solutions that improve the job search process for both job \nseekers and employers. We believe in transparency, trust, and collaboration, and we \nstrive to build strong relationships with our cu stomers and partners. Through our \nplatform, we aim to create a more engaged and fulfilled global workforce that drives \ninnovation and growth.  \n \nResponsibiliti

In [2]:
import pdfplumber
import PyPDF4
import re
import os
import sys
from typing import Callable, List, Tuple, Dict

from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

import time


from dotenv import load_dotenv

In [3]:
def extract_metadata_from_pdf(file_path: str) -> dict:
    with open(file_path, "rb") as pdf_file:
        reader = PyPDF4.PdfFileReader(pdf_file)  # Change this line
        metadata = reader.getDocumentInfo()
        return {
            "title": metadata.get("/Title", "").strip(),
            "author": metadata.get("/Author", "").strip(),
            "creation_date": metadata.get("/CreationDate", "").strip(),
        }


# def extract_pages_from_pdf(file_path: str) -> List[Tuple[int, str]]:
#     """
#     Extracts the text from each page of the PDF.

#     :param file_path: The path to the PDF file.
#     :return: A list of tuples containing the page number and the extracted text.
#     """
#     if not os.path.isfile(file_path):
#         raise FileNotFoundError(f"File not found: {file_path}")

#     with pdfplumber.open(file_path) as pdf:
#         pages = []
#         for page_num, page in enumerate(pdf.pages):
#             text = page.extract_text()
#             if text.strip():  # Check if extracted text is not empty
#                 pages.append((page_num + 1, text))
#     return pages


# def parse_pdf(file_path: str) -> Tuple[List[Tuple[int, str]], Dict[str, str]]:
#     """
#     Extracts the title and text from each page of the PDF.

#     :param file_path: The path to the PDF file.
#     :return: A tuple containing the title and a list of tuples with page numbers and extracted text.
#     """
#     if not os.path.isfile(file_path):
#         raise FileNotFoundError(f"File not found: {file_path}")

#     metadata = extract_metadata_from_pdf(file_path)
#     pages = extract_pages_from_pdf(file_path)

#     return pages, metadata


def merge_hyphenated_words(text: str) -> str:
    return re.sub(r"(\w)-\n(\w)", r"\1\2", text)


def fix_newlines(text: str) -> str:
    return re.sub(r"(?<!\n)\n(?!\n)", " ", text)


def remove_multiple_newlines(text: str) -> str:
    return re.sub(r"\n{2,}", "\n", text)


# def clean_text(
#     pages: List[Tuple[int, str]], cleaning_functions: List[Callable[[str], str]]
# ) -> List[Tuple[int, str]]:
#     cleaned_pages = []
#     for page_num, text in pages:
#         for cleaning_function in cleaning_functions:
#             text = cleaning_function(text)
#         cleaned_pages.append((page_num, text))
#     return cleaned_pages


# def text_to_docs(text: List[str], metadata: Dict[str, str]) -> List[Document]:
#     """Converts list of strings to a list of Documents with metadata."""
#     doc_chunks = []

#     for page_num, page in text:
#         text_splitter = RecursiveCharacterTextSplitter(
#             chunk_size=1000,
#             separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
#             chunk_overlap=200,
#         )
#         chunks = text_splitter.split_text(page)
#         for i, chunk in enumerate(chunks):
#             doc = Document(
#                 page_content=chunk,
#                 metadata={
#                     "page_number": page_num,
#                     "chunk": i,
#                     "source": f"p{page_num}-{i}",
#                     **metadata,
#                 },
#             )
#             doc_chunks.append(doc)

#     return doc_chunks

#Without pages
def parse_pdf_without_pages(file_path: str) -> Tuple[List[str], Dict[str, str]]:
    """
    Extracts the title and text from each page of the PDF.

    :param file_path: The path to the PDF file.
    :return: A tuple containing the title and a list of tuples with page numbers and extracted text.
    """
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    metadata = extract_metadata_from_pdf(file_path)
    pages = extract_text_from_pdf(file_path)

    return pages, metadata

# Without pages
def extract_text_from_pdf(file_path: str) -> List[str]:
    """
    Extracts the text from each page of the PDF.

    :param file_path: The path to the PDF file.
    :return: A list of tuples containing the page number and the extracted text.
    """
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    with pdfplumber.open(file_path) as pdf:
        pages = []
        text = ""
        for page_num, page in enumerate(pdf.pages):
            text += page.extract_text()
        if text.strip():  # Check if extracted text is not empty
            pages.append(text)
    return pages

# Without pages
def clean_text_without_pages(
    pages: List[str], cleaning_functions: List[Callable[[str], str]]
) -> List[str]:
    cleaned_pages = []
    for text in pages:
        for cleaning_function in cleaning_functions:
            text = cleaning_function(text)
        cleaned_pages.append(text)
    return cleaned_pages

# Without pages
def text_to_docs_without_pages(text: List[str], metadata: Dict[str, str]) -> List[Document]:
    """Converts list of strings to a list of Documents with metadata."""
    doc_chunks = []

    for page in text:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
            chunk_overlap=200,
        )
        chunks = text_splitter.split_text(page)
        for i, chunk in enumerate(chunks):
            doc = Document(
                page_content=chunk,
                metadata={
                    "chunk": i,
                    **metadata,
                },
            )
            doc_chunks.append(doc)

    return doc_chunks

In [4]:
if __name__ == "__main__":
    load_dotenv()

    start_time = time.time()

    # Step 1: Parse PDF
    file_path = "sample/resume_8.pdf"

    raw_pages, metadata = parse_pdf_without_pages(file_path)
    print(raw_pages)
    print(metadata)

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")

["Nguyen Thanh Phong\nGolang Developer\nDate of birth: April, 29th 1997\nGender: Male\nPhone: 0832210125\nEmail: thanhphong9718@gmail.com\nAddress: 5/3 Le Duc Tho Street, Go Vap District, Ho Chi Minh City\nEDUCATION\nSep 2015 - Sep 2019 HUTECH University\nMajor: Software Technology\nRanking: Good\nWORK EXPERIENCE\nJan 2023 - Present Phenikaa MaaS\nPosition: Golang Developer\nProject information:\n- MaaS Connect App (A comprehensive HR connection solution and\nface recognition for some client like: FE Credit, link:\nhttps://maasconnect.vn)\n- VF Staff XMĐ App (A smart application to help manage and organize\nevents effectively for VinFast employees)\nMain responsibilities:\n- Discuss with teams and client's team to come up with solutions\n- Read and analyze the requirement\n- Design and implement APIs\n- Write unit test using Go’s testing package\n- API integration\n- Fix bugs\nTechnology used:\n- Programming Languages: Golang\n- Lib: Gin, ORM (Gorm), Swagger, GoAdmin\n- Cache: Redis\n-

In [12]:
raw_pages

["FULL STACK ENGINEER\nAbout us\nQode is dedicated to helping technical talent around the world find meaningful careers\nthat match their skills and interests. Our platform provides a range of resources and\ntools that empower job seekers to take control of their careers and connect with top\nemployers across a variety of industries. We believe that every individual deserves to\nfind work that they're passionate about, and we are committed to making that vision a\nreality.\nQode's team of experienced professionals is passionate about creating a better world of\nwork by providing innovative solutions that improve the job search process for both job\nseekers and employers. We believe in transparency, trust, and collaboration, and we\nstrive to build strong relationships with our customers and partners. Through our\nplatform, we aim to create a more engaged and fulfilled global workforce that drives\ninnovation and growth.\nResponsibilities\n● Design and build a marketplace for tech talen

In [5]:
# Step 2: Create text chunks
cleaning_functions = [
    merge_hyphenated_words,
    fix_newlines,
    remove_multiple_newlines,
    ]
cleaned_text_pdf = clean_text_without_pages(raw_pages, cleaning_functions)
document_chunks = text_to_docs_without_pages(cleaned_text_pdf, metadata)



In [6]:
cleaned_text_pdf

["Nguyen Thanh Phong Golang Developer Date of birth: April, 29th 1997 Gender: Male Phone: 0832210125 Email: thanhphong9718@gmail.com Address: 5/3 Le Duc Tho Street, Go Vap District, Ho Chi Minh City EDUCATION Sep 2015 - Sep 2019 HUTECH University Major: Software Technology Ranking: Good WORK EXPERIENCE Jan 2023 - Present Phenikaa MaaS Position: Golang Developer Project information: - MaaS Connect App (A comprehensive HR connection solution and face recognition for some client like: FE Credit, link: https://maasconnect.vn) - VF Staff XMĐ App (A smart application to help manage and organize events effectively for VinFast employees) Main responsibilities: - Discuss with teams and client's team to come up with solutions - Read and analyze the requirement - Design and implement APIs - Write unit test using Go’s testing package - API integration - Fix bugs Technology used: - Programming Languages: Golang - Lib: Gin, ORM (Gorm), Swagger, GoAdmin - Cache: Redis - Database: Postgresql - Other: 

In [15]:
document_chunks

[Document(page_content="FULL STACK ENGINEER About us Qode is dedicated to helping technical talent around the world find meaningful careers that match their skills and interests. Our platform provides a range of resources and tools that empower job seekers to take control of their careers and connect with top employers across a variety of industries. We believe that every individual deserves to find work that they're passionate about, and we are committed to making that vision a reality. Qode's team of experienced professionals is passionate about creating a better world of work by providing innovative solutions that improve the job search process for both job seekers and employers. We believe in transparency, trust, and collaboration, and we strive to build strong relationships with our customers and partners. Through our platform, we aim to create a more engaged and fulfilled global workforce that drives innovation and growth. Responsibilities ● Design and build a marketplace for tec

In [16]:
    # Optional: Reduce embedding cost by only using the first 23 pages
#     document_chunks = document_chunks[:70]

    # Step 3 + 4: Generate embeddings and store them in DB
embeddings = OpenAIEmbeddings()

vector_store = Chroma.from_documents(
        document_chunks,
        embeddings,
        collection_name="jd_1",
        persist_directory="data/chroma/jd",
    )

    # Save DB locally
vector_store.persist()

In [21]:
# from langchain.vectorstores import FAISS

# faiss_store = FAISS.from_documents(
#     document_chunks,
#     embeddings,
# )

# # Save DB locally
# faiss_store.save_local("data/faiss/resume_sample_1")



In [19]:
!pip install faiss-cpu

  Using cached faiss_cpu-1.7.4-cp310-cp310-macosx_11_0_arm64.whl (2.7 MB)
